In [1]:
import cifar_gpu_fun
reload(cifar_gpu_fun)
from cifar_gpu_fun import *
from numba import jit
import bcd
%pylab inline

Using gpu device 0: Tesla K20c (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 5103)
/data/vaishaal/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)
/data/vaishaal/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


#### Define Some Constants 

In [7]:
NUM_FEATURE_BATCHES = 4
DATA_BATCH_SIZE=(1280)
FEATURE_BATCH_SIZE=(1024)
NUM_TRAIN = 50000
NUM_TEST = 10000
NUM_CLASSES = 10
NUM_ENSEMBLES = 8
NUM_FEATURES = FEATURE_BATCH_SIZE * NUM_FEATURE_BATCHES * 8
POOL_TYPE ='avg'
FILTER_GEN ='empirical'
BANDWIDTH = 1.0
LAMBDAS = [1.0, 0.1, 0.01, 0.001]
CUDA_CONVNET = True
SCALE = 55.0
BIAS = 1.25
MIN_VAR_TOL = 1e-4
TOT_FEAT = FEATURE_BATCH_SIZE*NUM_FEATURE_BATCHES
NAN = float('nan')

In [8]:
np.random.seed(10)                                                                                                                                                                   
(XTrain, labelsTrain), (XTest, labelsTest) = load_cifar_processed()
if FILTER_GEN == 'gaussian':
    filter_gen = make_gaussian_filter_gen(BANDWIDTH)
elif FILTER_GEN == 'empirical':
    patches = patchify_all_imgs(XTrain, (6,6), pad=False)
    filter_gen = make_empirical_filter_gen(patches, labelsTrain, MIN_VAR_TOL)
elif FILTER_GEN == 'gaussian_dropout':
    filter_gen = make_gaussian_dropout_filter_gen(BANDWIDTH)
else:
    raise Exception('Unknown FILTER_GEN value')

(50000, 3, 32, 32)


In [9]:
@jit(nopython=True)
def rbf(K, gamma):
    for x in range(K.shape[0]):
        for y in range(K.shape[1]):
            K[x,y] = math.exp(gamma*K[x,y])
            
    return K

@jit(nopython=True)
def arccos(K, XTrainNorms, XTestNorms):
    for x in range(K.shape[0]):
        for y in range(K.shape[1]):
            testNorm = np.sqrt(XTestNorms[x])
            trainNorm = np.sqrt(XTrainNorms[y]) 
            if (trainNorm == 0):
                trainNorm = 1
            if (testNorm == 0):
                testNorm = 1 
            cos = min(K[x,y]/(testNorm*trainNorm), 1)
            theta = np.arccos(cos)
            J = math.sin(theta) - (math.pi - theta)*math.cos(theta)
            K[x,y] = 1.0/math.pi * J
    return K
    


def computeRBFGramMatrix(K, XTrainNorms, XTestNorms, gamma=1, gamma_sample=10000):
    
    XTrainNorms = XTrainNorms.reshape(XTrainNorms.shape[0], 1)
    XTestNorms = XTestNorms.reshape(XTestNorms.shape[0], 1)
    print "TURNING K -> DISTANCE"
    K *= -2
    K += XTrainNorms.T
    K += XTestNorms 
    if (gamma == None):
        print "Calculating gamma"
        samples = numpy.random.choice(K.shape[0], gamma_sample*2, replace=False)
        x1 = samples[:gamma_sample]
        x2 = samples[gamma_sample:]
        sample_d = K[x1, x2]
        print "Sample d shape ", sample_d.shape
        median = numpy.median(np.sqrt(sample_d))
        gamma = 2.0/median
        print gamma
    gamma = -1.0 * gamma

    print "Computing RBF"
    return rbf(K, gamma), -1.0*gamma


    

In [5]:
models = []
train_predictions = []
test_predictions = []

In [ ]:
for _ in range(NUM_ENSEMBLES)
    KTrain, KTest, normsTrain, normsTest = featurizeTrainAndEvaluateDualModelAsync(XTrain, XTest, 
                                                                         labelsTrain, labelsTest, filter_gen,
                                                                         num_feature_batches=NUM_FEATURE_BATCHES, pool_size=10+pool_offset, pool_stride=10)
    KTrain /= float(TOT_FEAT)
    KTest /= float(TOT_FEAT)
    normsTrain /= float(TOT_FEAT)
    normsTest /= float(TOT_FEAT)
    result = {}
    trainAndEvaluateDualModel(KTrain, KTest, labelsTrain, labelsTest, reg=reg)
        print pool_offset, reg, acc
        result[reg] = acc
    results[pool_offset] = result


RELOADING MOTHER FUCKER 3
9
Convolving features
('Pool Size ', 12)
('Pool Stride', 10)
('out size: ', 3, 'x', 3)
Out Shape  3 x 3 x 2048
FEATURE BATCH # 0 DATA BATCH # 0  SIZE IS  1280
CHILD Process Spun yupyup
FEATURE BATCH # 0 DATA BATCH # 1  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 2  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 3  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 4  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 5  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 6  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 7  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 8  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 9  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 10  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 11  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 12  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 13  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 14  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 15  SIZE IS  1280
FEATURE BATCH # 0 DATA BATCH # 16  SIZE IS  1280
FEATURE BATCH 

In [ ]:
for pool_offset in results.keys():
    for reg in results[pool_offset].keys():
        train, test = results[pool_offset][reg]
        print pool_offset, train, test, reg

In [109]:
KTrainRBF = KTrain.copy()
KTestRBF = KTest.copy()
KTrainRBF, train_gamma = computeRBFGramMatrix(KTrainRBF, normsTrain, normsTrain, gamma=0.005)
KTestRBF, _ = computeRBFGramMatrix(KTestRBF, normsTrain, normsTest, gamma=0.005)

TURNING K -> DISTANCE
Computing RBF
TURNING K -> DISTANCE
Computing RBF


In [111]:
%time acc2 = trainAndEvaluateDualModel(KTrainRBF, KTestRBF, labelsTrain, labelsTest, reg=1e-4)
print acc2

Learning Dual Model better
REG IS  0.0001
CPU times: user 25min 45s, sys: 12.7 s, total: 25min 58s
Wall time: 1min 29s
(0.99878, 0.83709999999999996)


In [51]:
KT[0,0]

11.099676132202148

In [19]:
KTrainACOS

array([[  9.88883550e+02,  -1.12112873e+01,  -2.49319883e+00, ...,
         -3.50704977e+00,  -6.86583103e+00,  -8.71419617e+00],
       [ -1.12112873e+01,   9.79671277e+02,  -3.35546961e+00, ...,
         -4.66814909e+00,  -1.00538406e+01,  -1.13506800e+01],
       [ -2.49319883e+00,  -3.35546961e+00,   9.97998944e+02, ...,
         -2.63581442e+00,  -6.16682802e-01,  -3.25858016e+00],
       ..., 
       [ -3.50704977e+00,  -4.66814909e+00,  -2.63581442e+00, ...,
          9.95809525e+02,  -1.03835189e+00,  -4.47105405e+00],
       [ -6.86583103e+00,  -1.00538406e+01,  -6.16682802e-01, ...,
         -1.03835189e+00,   9.88047484e+02,  -6.16753853e+00],
       [ -8.71419617e+00,  -1.13506800e+01,  -3.25858016e+00, ...,
         -4.47105405e+00,  -6.16753853e+00,   9.86328712e+02]])

In [167]:
del KTestRBF

In [168]:
del KTestACOS

In [169]:
del KTrainACOS

In [ ]:
gc.collect()